# Get to Know a Dataset: Canoe

This notebook serves as a guided tour of the [CANOE](https://registry.opendata.aws/canoe) dataset. More usage examples, tutorials, and documentation for this dataset and others can be found at the [Registry of Open Data on AWS](https://registry.opendata.aws/).

### Q: How have you organized your dataset? Help us understand the key prefix structure of your S3 bucket.



### Q: What data formats are present in your dataset? What kinds of data are stored using these formats? Can you give any advice for how you work with these data formats?


### Q: A picture is worth a thousand words. Show us a visual (or several!) from your dataset that either illustrates something informative about your dataset, or that you think might excite someone to dig in further.



### Q: What is one question that you have answered using these data? Can you show us how you came to that answer?



### Q: What is one unanswered question that you think could be answered using these data? Do you have any recommendations or advice for someone wanting to answer this question?
